# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

In [1]:
"""
- This chatbot application is made specialized on 'New York food scrap drop off sites'. For a few of the factors such as site_area, operating_hours and what's not permitted.
- Demonstrates the use of custom dataset with an LLM by using RAG (Retrieval Augmented Generation).
- I have taken the dataset named "nyc_food_scrap_drop_off_sites.csv" provided in the project workspace under "data" folder.
- As mentioned here in https://platform.openai.com/docs/models#gpt-3-5-turbo "gpt-3.5-turbo-instruct" has the knowledge cutoff of Sep 2021, hence by using this dataset we could feed latest context to prompts.
"""

'\n- This chatbot application is made specialized on \'New York food scrap drop off sites\'. For a few of the factors such as site_area, operating_hours and what\'s not permitted.\n- Demonstrates the use of custom dataset with an LLM by using RAG (Retrieval Augmented Generation).\n- I have taken the dataset named "nyc_food_scrap_drop_off_sites.csv" provided in the project workspace under "data" folder.\n- As mentioned here in https://platform.openai.com/docs/models#gpt-3-5-turbo "gpt-3.5-turbo-instruct" has the knowledge cutoff of Sep 2021, hence by using this dataset we could feed latest context to prompts.\n'

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [2]:
import pandas as pd

results_df = pd.read_csv("data/nyc_food_scrap_drop_off_sites.csv")
results_df.info()

print(results_df.iloc[0]["borough"])
print(results_df.iloc[0]["ntaname"])
print(results_df.iloc[0]["food_scrap_drop_off_site"])
print(results_df.iloc[0]["operation_day_hours"])
print(results_df.iloc[0]["notes"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   576 non-null    int64  
 1   borough                      576 non-null    object 
 2   ntaname                      576 non-null    object 
 3   food_scrap_drop_off_site     576 non-null    object 
 4   location                     335 non-null    object 
 5   hosted_by                    571 non-null    object 
 6   open_months                  576 non-null    object 
 7   operation_day_hours          576 non-null    object 
 8   website                      541 non-null    object 
 9   borocd                       576 non-null    int64  
 10  councildist                  576 non-null    int64  
 11  latitude                     335 non-null    float64
 12  longitude                    335 non-null    float64
 13  precinct            

In [3]:
results_text = []
not_accepted_row_count_min = 192
for (i, row) in results_df.iterrows():
    food_scrap_drop_off_site = row['food_scrap_drop_off_site']
    # clean food_scrap_drop_off_site text
    if ":" in row["food_scrap_drop_off_site"]:
        food_scrap_drop_off_site = row['food_scrap_drop_off_site'].split(': ')[1]

    notes = row['notes']
    if not isinstance(notes, str):
        continue
    # Ensure to have some rows with not_accepted info in the notes
    if 'Not accepted' not in row['notes'] and not_accepted_row_count_min > 0:
        continue
    else:
        not_accepted_row_count_min -= 1

    results_text.append({
        "text":
            f"area - {row['borough']} | "
            f"suburb - {row['ntaname']} | "
            f"food_scrap_drop_off_site - {food_scrap_drop_off_site} | "
            f"operation_day_hours - {row['operation_day_hours']} | "
            f"accepted / not_accepted - {notes}"
    })

df = pd.DataFrame.from_records(results_text)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    236 non-null    object
dtypes: object(1)
memory usage: 2.0+ KB


In [4]:
df.head()

,text
0,area - Queens | suburb - Astoria (North)-Ditma...
1,area - Queens | suburb - Astoria (Central) | f...
2,area - Queens | suburb - Queensbridge-Ravenswo...
3,area - Brooklyn | suburb - Crown Heights (Nort...
4,area - Queens | suburb - Long Island City-Hunt...


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [5]:
import openai

openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "YOUR API KEY"

In [6]:
que_1 = "List some of the food scarp drop off sites in Brooklyn area which operate 24/7?"
prompt_1 = f"""
Question: {que_1}
Answer:
"""
initial_answer_1 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt_1,
    max_tokens=200
)["choices"][0]["text"].strip()
print("\n" + prompt_1 + "\n" + initial_answer_1)



Question: List some of the food scarp drop off sites in Brooklyn area which operate 24/7?
Answer:

1. Prospect Heights Community Farm (256 St Marks Ave)
2. Red Hook Community Farm (580 Columbia St)
3. Carroll Gardens Association (201 Columbia Street)
4. East New York Farms! (613 New Lots Avenue)
5. Bushwick City Farm (354 Johnson Ave)
6. Wyckoff Farmhouse Museum (5816 Clarendon Rd)
7. Project EATS at E New York Farms! (613 New Lots Ave)
8. East New York Farms! (613 New Lots Ave)
9. Smorgasburg at Prospect Park (Prospect Park West & 4th Street)
10. New Complaints (244 E Broadway)


In [7]:
que_2 = "What items are not accepted in the bins at some of the locations?"
prompt_2 = f"""
Question: {que_2}
Answer:
"""
initial_answer_2 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt_2,
    max_tokens=200
)["choices"][0]["text"].strip()
print("\n" + prompt_2 + "\n" + initial_answer_2)



Question: What items are not accepted in the bins at some of the locations?
Answer:

1. Food waste (such as leftover food scraps)
2. Liquids and wet materials
3. Hazardous waste (such as batteries, chemicals, or electronics)
4. Medical waste (such as used needles)
5. Large or bulky items (such as furniture or appliances)
6. Styrofoam and plastic bags
7. Clothing or textiles
8. Garden waste or yard trimmings
9. Construction or demolition waste
10. Paint or other liquid-based products


In [8]:
# Generating Embeddings
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i + batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )

    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,area - Queens | suburb - Astoria (North)-Ditma...,"[-0.0010825099889189005, -0.007952152751386166..."
1,area - Queens | suburb - Astoria (Central) | f...,"[0.005009927321225405, -0.011369999498128891, ..."
2,area - Queens | suburb - Queensbridge-Ravenswo...,"[0.0039951554499566555, -0.018161669373512268,..."
3,area - Brooklyn | suburb - Crown Heights (Nort...,"[0.015915587544441223, -0.019540881738066673, ..."
4,area - Queens | suburb - Long Island City-Hunt...,"[0.001311104861088097, -0.01317282859236002, -..."
...,...,...
231,area - Brooklyn | suburb - Kensington | food_s...,"[0.0033036507666110992, -0.018176931887865067,..."
232,area - Queens | suburb - Old Astoria-Hallets P...,"[0.012196875177323818, -0.008693517185747623, ..."
233,area - Brooklyn | suburb - Crown Heights (Nort...,"[0.008765354752540588, -0.015816349536180496, ..."
234,area - Brooklyn | suburb - Windsor Terrace-Sou...,"[0.010718269273638725, -0.028937920928001404, ..."


In [9]:
df.to_csv("embeddings.csv")

In [10]:
# Create a Function that Finds Related Pieces of Text for a Given Question
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """

    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)

    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )

    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy


get_rows_sorted_by_relevance(que_1, df)
get_rows_sorted_by_relevance(que_2, df)

,text,embeddings,distances
202,area - Manhattan | suburb - Financial District...,"[0.006152546498924494, -0.017554989084601402, ...",0.180481
233,area - Brooklyn | suburb - Crown Heights (Nort...,"[0.008765354752540588, -0.015816349536180496, ...",0.184575
216,area - Brooklyn | suburb - Bedford-Stuyvesant ...,"[0.004639037884771824, -0.009155277162790298, ...",0.185346
214,area - Manhattan | suburb - Washington Heights...,"[0.007210155017673969, -0.0002376664779148996,...",0.185572
210,area - Brooklyn | suburb - Bushwick (West) | f...,"[-0.0022310935892164707, -0.01732621155679226,...",0.186341
...,...,...,...
23,area - Queens | suburb - Jamaica | food_scrap_...,"[-0.0017969474429264665, -0.01404793094843626,...",0.232709
98,area - Queens | suburb - Rockaway Beach-Arvern...,"[-0.0009301416575908661, -0.036067284643650055...",0.232879
143,area - Queens | suburb - Rockaway Beach-Arvern...,"[-0.01305895671248436, -0.028953880071640015, ...",0.234847
165,area - Queens | suburb - Douglaston-Little Nec...,"[-0.004628377966582775, -0.011639009229838848,...",0.236809


In [11]:
# Create a Function that Composes a Text Prompt
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                          len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)


print(create_prompt(que_1, df, 1800))
print(create_prompt(que_2, df, 1800))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

area - Brooklyn | suburb - East New York (North) | food_scrap_drop_off_site - St. John Community Garden | operation_day_hours - 24/7 (Start Time: 24/7 - End Time:  24/7) | accepted / not_accepted - Not accepted: meat, bones, or dairy

###

area - Brooklyn | suburb - East New York-New Lots | food_scrap_drop_off_site - UCC Youth Farm | operation_day_hours - 24/7 (Start Time: 24/7 - End Time:  24/7) | accepted / not_accepted - Not accepted: meat, bones, or dairy

###

area - Brooklyn | suburb - East New York (North) | food_scrap_drop_off_site - Ashford Learning Garden | operation_day_hours - 24/7 (Start Time: 24/7 - End Time:  24/7) | accepted / not_accepted - Not accepted: meat, bones, or dairy

###

area - Brooklyn | suburb - East New York (North) | food_scrap_drop_off_site - Concerned Residents of Barbey Street Community Garden | operation_day_hour

In [12]:
# Create a Function that Answers a Question
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"


def answer_question(
        question, df, max_prompt_tokens=1800, max_answer_tokens=500
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model

    If the model produces an error, return an empty string
    """

    prompt = create_prompt(question, df, max_prompt_tokens)

    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [13]:
custom_answer_1 = answer_question(que_1, df)

In [14]:
print("Custom Answer 1: " + custom_answer_1)

Custom Answer 1: St. John Community Garden, Ashford Learning Garden, Concerned Residents of Barbey Street Community Garden, Tompkins Avenue & Putnam Avenue, Rochester Avenue & St. Johns Place, EL Garden, NE Corner of Knickerbocker Avenue & Putnam Avenue, East New York-New Lots, Rockaway Avenue Community Compost Site, and SW Corner of Tompkins Avenue & Putnam Avenue all operate 24/7.


### Question 2

In [15]:
custom_answer_2 = answer_question(que_2, df)

In [16]:
print("Custom Answer 2: " + custom_answer_2)

Custom Answer 2: Meat, bones, or dairy


In [17]:
# Comparisons of answers with and without context

print(f"""
{que_1}

Original Answer: {initial_answer_1}
\n\nCustom Answer:   {custom_answer_1}

---

{que_2}
Original Answer: {initial_answer_2}
\n\nCustom Answer:   {custom_answer_2}
""")


List some of the food scarp drop off sites in Brooklyn area which operate 24/7?

Original Answer: 1. Prospect Heights Community Farm (256 St Marks Ave)
2. Red Hook Community Farm (580 Columbia St)
3. Carroll Gardens Association (201 Columbia Street)
4. East New York Farms! (613 New Lots Avenue)
5. Bushwick City Farm (354 Johnson Ave)
6. Wyckoff Farmhouse Museum (5816 Clarendon Rd)
7. Project EATS at E New York Farms! (613 New Lots Ave)
8. East New York Farms! (613 New Lots Ave)
9. Smorgasburg at Prospect Park (Prospect Park West & 4th Street)
10. New Complaints (244 E Broadway)


Custom Answer:   St. John Community Garden, Ashford Learning Garden, Concerned Residents of Barbey Street Community Garden, Tompkins Avenue & Putnam Avenue, Rochester Avenue & St. Johns Place, EL Garden, NE Corner of Knickerbocker Avenue & Putnam Avenue, East New York-New Lots, Rockaway Avenue Community Compost Site, and SW Corner of Tompkins Avenue & Putnam Avenue all operate 24/7.

---

What items are not a

In [18]:
"""
Hence, we could see the changes from original answers to the customised ones.
"""

'\nHence, we could see the changes from original answers to the customised ones.\n'